In [16]:
import os
import string
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns
from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

In [17]:
# from ..src.preprocess_data import is_climate_feat

In [18]:
DIR = "../../data/"
SUBWAYUS = "Subway USA/subway_usa_"

In [19]:
demographic = pd.read_csv(DIR + SUBWAYUS + "demographic_variables.csv")
stores = pd.read_csv(DIR + SUBWAYUS + "stores.csv")
poi_variables = pd.read_csv(DIR + SUBWAYUS + "poi_variables.csv")
sister = pd.read_csv(DIR + SUBWAYUS + "competition_sister_variables.csv")
trade_area = pd.read_csv(DIR + SUBWAYUS + "trade_area_variables.csv")

In [23]:
def is_climate_feat(feat):
    return "avgmax" in feat or "temp" in feat or feat == "precip" or feat == "snowfall"

def is_sport_venue(feat):
    return "sports_venues" in feat

def drop_specific_columns(df):
    all_cols = df.columns.tolist()
    keep_columns = []
    for col in all_cols:
        if "centerxy" in col:
            if "full" not in col and "effective" not in col:
                # cols_to_remove.append(col)
                continue
        elif is_climate_feat(col):
            # cols_to_remove.append(col)
            continue
        elif is_sport_venue(col):
            # cols_to_remove.append(col)
            continue
        elif col.startswith('edu') and not col.startswith('edu_bachplus_p'):
            continue
        else:
            keep_columns.append(col)
    print(f'----- Removing {len(all_cols) - len(keep_columns)} columns -----')
    reduced_df = df[keep_columns]
    return reduced_df

In [21]:
merged = stores.merge(
    poi_variables, on="store"
# ).merge(
#     demographic, on="store"
).merge(
    trade_area, on="store"
)

In [24]:
merged = drop_specific_columns(merged)

----- Removing 52 columns -----


In [ ]:
# def simplify_edu(df):
#     edu_columns = filter(lambda x: x.startswith('edu') and not x.startswith('edu_bachplus_p'), df.columns.tolist())
#     print(f'----- Removing {len(set(edu_columns))} columns -----')
#     keep_edu_columns = list(set(df.columns.tolist()) - set(edu_columns))
#     return df[keep_edu_columns]

In [26]:
def process_percent_non_percent_df(df):
    all_cols = df.columns.tolist()
    percent_feats = [col for col in all_cols if "_p_" in col]
    non_percent_feats = list(set(all_cols) - set(percent_feats))
    overlapping_feats = []
    for feat in percent_feats:
        remove_percent_feat = "_".join(feat.split("_p_"))
        if remove_percent_feat in non_percent_feats:
            overlapping_feats.append(remove_percent_feat)
    no_overlap = list(set(non_percent_feats) - set(overlapping_feats))
    reduced_feats = percent_feats
    reduced_feats.extend(no_overlap)
    reduced_feats.sort()
    reduced_number = len(all_cols) - len(reduced_feats)
    print(f'----- Removing {reduced_number} columns -----')
    reduced_df = df[reduced_feats]
    return reduced_df

merged = process_percent_non_percent_df(merged)

----- Removing 47 columns -----


In [28]:
train_df, test_df = train_test_split(merged, test_size=0.1, random_state=42)

In [29]:
train_index = train_df['store']
test_index = test_df['store']

In [30]:
drop_features = [
    "store", 
    "longitude", 
    "latitude", 
    # "__store_latitude",
    # "__store_longitude",
    # "__batch_group",
    # "__errors",
    # "__store_bg",
    # "__success",
]

ordinal_features_oth = [
    "market_size",
    "store_density",
]
ordering_ordinal_oth = [
    ["Very Large Metro (1)", "Large Metro (2)", "Large City (3)", "Medium City (4)", "Small City (5)", "Small Town (6)", "Small Community (7)", ""],
    ["Rural", "Exurban", "Suburban", "Light Suburban", "Light Urban", "Urban", "Super Urban"],
]
categorical_features = ["cbsa_name", "dma_name", "censusdivision", "censusregion"]

numeric_features = list(set(merged.select_dtypes(include=np.number).columns.tolist()) - set(drop_features))

In [31]:
numeric_transformer = make_pipeline(
    SimpleImputer(strategy="median"), 
    StandardScaler()
)

ordinal_transformer_oth = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(categories=ordering_ordinal_oth),
)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore", sparse=False),
)

preprocessor = make_column_transformer(
    ("drop", drop_features + categorical_features),
    (numeric_transformer, numeric_features),
    (ordinal_transformer_oth, ordinal_features_oth),
    # (categorical_transformer, categorical_features),
)

In [32]:
train_df.columns.tolist()

['age0018_p_ta',
 'age65pl_p_ta',
 'age85pl_p_ta',
 'asian_p_ta',
 'avg_faminc_ta',
 'avghhinc_ta',
 'banks_1mi',
 'banks_2mi',
 'banks_3mi',
 'banks_5mi',
 'black_p_ta',
 'boomer_p_ta',
 'cbsa_name',
 'censusdivision',
 'censusregion',
 'com0002_p_ta',
 'com0205_p_ta',
 'com0508_p_ta',
 'com0811_p_ta',
 'com12pl_p_ta',
 'crime_total_index_ta',
 'daypop_dens_ta',
 'disposable_inc_avg_ta',
 'dma_name',
 'dmm_count_1mi',
 'dmm_count_2mi',
 'dmm_count_3mi',
 'dmm_count_5mi',
 'dmm_gla_1mi',
 'dmm_gla_2mi',
 'dmm_gla_3mi',
 'dmm_gla_5mi',
 'dmm_nearest_dist',
 'dtpop_children_at_home_p_ta',
 'dtpop_homemakers_p_ta',
 'dtpop_retired_disabled_p_ta',
 'dtpop_students_9th_12th_p_ta',
 'dtpop_students_p_ta',
 'dtpop_students_post_secondary_p_ta',
 'dtpop_students_prek_8th_p_ta',
 'dtpop_ta',
 'dtpop_unemployed_p_ta',
 'dtpop_work_at_home_p_ta',
 'edu_bachplus_p_ta',
 'emp_accommodation_foodserv_p_ta',
 'emp_admin_support_p_ta',
 'emp_arts_entertainment_p_ta',
 'emp_construction_p_ta',
 'emp_edu

In [33]:
preprocessor.fit(train_df)

ColumnTransformer(transformers=[('drop', 'drop',
                                 ['store', 'longitude', 'latitude', 'cbsa_name',
                                  'dma_name', 'censusdivision',
                                  'censusregion']),
                                ('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['hh_type_nonfam_p_ta', 'emp_management_p_ta',
                                  'hhinc150pl_p_ta',
                                  'emp_finance_insurance_p_ta',
                                  'pl...
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Very '
                                                                              'Large '
                                                                              'Metro '
                                                                              '(1)',
                                                                              'Large '
                                                                              'Metro '
                                                                              '(2)',
                                                                              'Large '
                                                                              'City '
                                                                              '(3)',
                                                                              'Medium '
                                                                              'City '
                                                                              '(4)',
                                                                              'Small '
                                                                              'City '
                                                                              '(5)',
                                                                              'Small '
                                                                              'Town '
                                                                              '(6)',
                                                                              'Small '
                                                                              'Community '
                                                                              '(7)',
                                                                              ''],
                                                                             ['Rural',
                                                                              'Exurban',
                                                                              'Suburban',
                                                                              'Light '
                                                                              'Suburban',
                                                                              'Light '
                                                                              'Urban',
                                                                              'Urban',
                                                                              'Super '
                                                                              'Urban']]))]),
                                 ['market_size', 'store_density'])])

In [34]:
transformed_train = preprocessor.transform(train_df)
transformed_test = preprocessor.transform(test_df)

In [35]:
column_names = (
    numeric_features
    + preprocessor.named_transformers_['pipeline-2']['ordinalencoder'].get_feature_names_out().tolist()
    # + preprocessor.named_transformers_['pipeline-3']['onehotencoder'].get_feature_names_out().tolist()
)
len(column_names)

205

In [36]:
transformed_train_df = pd.DataFrame(transformed_train, columns=column_names, index=train_index)
transformed_test_df = pd.DataFrame(transformed_test, columns=column_names, index=test_index)
transformed_train_df

,hh_type_nonfam_p_ta,emp_management_p_ta,hhinc150pl_p_ta,emp_finance_insurance_p_ta,places_of_worship_1mi,ipeds_postsecondary_schools_total_enrollment_2mi,emp_educ_services_p_ta,banks_5mi,inrix_overnight_ew,pop_seasonal_ta,...,dmm_count_2mi,hh_5vehicle_p_ta,white_p_ta,hh_3vehicle_p_ta,osm_highway_exits_count_3mi,nces_private_schools_2mi,hh_1pers_p_ta,popgrpy_ta,x0,x1
store,,,,,,,,,,,,,,,,,,,,,
6150300,-1.109287,-0.504268,-1.197578,-0.739929,0.119592,-0.300746,2.422405,-0.527041,-0.203609,-0.258713,...,-0.060308,0.100148,-1.749451,-0.283036,0.047151,0.032203,-0.577592,-0.622184,2.0,3.0
3784100,-0.371554,0.036547,-0.483549,-0.817860,-0.432111,-0.300746,0.878244,-0.285219,0.617314,-0.317983,...,-0.060308,-0.802474,0.029789,-0.057683,-0.635362,0.231835,-0.636700,0.968952,2.0,1.0
1192500,0.366179,-0.156602,0.235945,-0.705293,-0.050163,-0.300746,-0.533329,-0.342118,-0.331131,0.003411,...,0.535134,-0.209511,0.457056,-0.477305,0.047151,-0.367062,0.170248,-0.349460,2.0,1.0
449400,0.143518,-0.504268,-0.754042,-0.618703,0.162030,-0.297594,-0.282706,-0.327893,-0.767496,-0.136834,...,1.130575,-0.538935,0.363884,-0.232526,-0.740364,-0.367062,0.202371,-0.199782,4.0,1.0
2292700,-0.645186,-0.388379,0.435399,-0.185753,0.883488,0.093580,0.077867,0.497147,1.264887,-0.308800,...,-0.060308,2.610359,-1.577531,0.931150,-0.425358,0.631100,-1.572142,-0.350559,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440100,-0.103288,-0.504268,-0.931638,-0.558090,-0.219918,-0.300746,-0.691788,-0.370568,-0.335116,-0.260383,...,-0.655749,-0.894713,1.167739,0.023910,-0.005350,-0.367062,0.548022,-1.169971,4.0,0.0
6487000,-0.578119,-0.504268,-0.568249,-0.471500,-0.219918,-0.090717,-0.735445,-0.214095,-0.576213,-0.271235,...,1.130575,-0.407165,-0.164352,0.062764,-0.057851,-0.367062,0.077731,0.035201,4.0,1.0
2493300,1.028797,0.558046,0.112993,-0.488818,-0.389673,-0.111978,-0.303726,-0.043397,0.190912,-0.094260,...,0.535134,-0.598231,1.018819,-0.372400,0.204654,-0.167430,0.386119,-0.340691,1.0,1.0


In [37]:
print(column_names.sort())

None


In [40]:
column_names[:50]

['age0018_p_ta',
 'age65pl_p_ta',
 'age85pl_p_ta',
 'asian_p_ta',
 'avg_faminc_ta',
 'avghhinc_ta',
 'banks_1mi',
 'banks_2mi',
 'banks_3mi',
 'banks_5mi',
 'black_p_ta',
 'boomer_p_ta',
 'com0002_p_ta',
 'com0205_p_ta',
 'com0508_p_ta',
 'com0811_p_ta',
 'com12pl_p_ta',
 'crime_total_index_ta',
 'daypop_dens_ta',
 'disposable_inc_avg_ta',
 'dmm_count_1mi',
 'dmm_count_2mi',
 'dmm_count_3mi',
 'dmm_count_5mi',
 'dmm_gla_1mi',
 'dmm_gla_2mi',
 'dmm_gla_3mi',
 'dmm_gla_5mi',
 'dmm_nearest_dist',
 'dtpop_children_at_home_p_ta',
 'dtpop_homemakers_p_ta',
 'dtpop_retired_disabled_p_ta',
 'dtpop_students_9th_12th_p_ta',
 'dtpop_students_p_ta',
 'dtpop_students_post_secondary_p_ta',
 'dtpop_students_prek_8th_p_ta',
 'dtpop_ta',
 'dtpop_unemployed_p_ta',
 'dtpop_work_at_home_p_ta',
 'edu_bachplus_p_ta',
 'emp_accommodation_foodserv_p_ta',
 'emp_admin_support_p_ta',
 'emp_arts_entertainment_p_ta',
 'emp_construction_p_ta',
 'emp_educ_services_p_ta',
 'emp_finance_insurance_p_ta',
 'emp_healthca

In [41]:
transformed_train_df.to_csv(DIR + SUBWAYUS + "train.csv")
transformed_test_df.to_csv(DIR + SUBWAYUS + "test.csv")